# Using Dlib based on CNN

Detects faces at odd angles.

Architecture of CNN used in Dlib
================================

 template <template <int,template<typename>class,int,typename> class block, int N, template<typename>class BN, typename SUBNET>
    using residual = add_prev1<block<N,BN,1,tag1<SUBNET>>>;

    template <template <int,template<typename>class,int,typename> class block, int N, template<typename>class BN, typename SUBNET>
    using residual_down = add_prev2<avg_pool<2,2,2,2,skip1<tag2<block<N,BN,2,tag1<SUBNET>>>>>>;

    template <int N, template <typename> class BN, int stride, typename SUBNET> 
    using block  = BN<con<N,3,3,1,1,relu<BN<con<N,3,3,stride,stride,SUBNET>>>>>;

    template <int N, typename SUBNET> using ares      = relu<residual<block,N,affine,SUBNET>>;
    template <int N, typename SUBNET> using ares_down = relu<residual_down<block,N,affine,SUBNET>>;

    template <typename SUBNET> using alevel0 = ares_down<256,SUBNET>;
    template <typename SUBNET> using alevel1 = ares<256,ares<256,ares_down<256,SUBNET>>>;
    template <typename SUBNET> using alevel2 = ares<128,ares<128,ares_down<128,SUBNET>>>;
    template <typename SUBNET> using alevel3 = ares<64,ares<64,ares<64,ares_down<64,SUBNET>>>>;
    template <typename SUBNET> using alevel4 = ares<32,ares<32,ares<32,SUBNET>>>;

    using anet_type = loss_metric<fc_no_bias<128,avg_pool_everything<
                                alevel0<
                                alevel1<
                                alevel2<
                                alevel3<
                                alevel4<
                                max_pool<3,3,2,2,relu<affine<con<32,7,7,2,2,
                                input_rgb_image_sized<150>
                                >>>>>>>>>>>>;



In [2]:
import dlib
import cv2
import numpy as np
print("imported")

imported


# Model
 ‘dlib’ in python uses these weights and detect images,but dlib doesn’t provide this detector as in-built. We must download and provide to ‘dlib’ classes for extracting face.

In [3]:
detector = dlib.cnn_face_detection_model_v1("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face Detection\\dlib\\hog\\mmod_human_face_detector.dat")
print("done")

done


In [7]:
cap = cv2.VideoCapture(0)
flag = 0
import time
while True:
    # Capture frame-by-frame

    _, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (0, 0), fx=0.25, fy=0.25)

    rects = detector(gray, 0)

    for (i, rect) in enumerate(rects):

        x1 = rect.rect.left() *4
        y1 = rect.rect.top() *4
        x2 = rect.rect.right() *4
        y2 = rect.rect.bottom() *4

        # Rectangle around the face
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)


    # Display the video output
    cv2.imshow('Video', frame)


    # Quit video by typing Q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

 # Comments 

Sloww, doesnt detect unless complete face in fov